In [ ]:
pip install pandas requests tqdm

In [ ]:
import os
import pandas as pd
import requests
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm

# --- 1. CONFIGURAZIONE PERCORSI E PARAMETRI ---
# Sostituisci con il nome esatto del file CSV che hai scaricato
CSV_PATH = "data/raw/productos.csv" 
OUTPUT_DIR = "data/images/products/"
# Quanti download paralleli fare. 20 è un numero sicuro, ma se siete 
# su una rete veloce (es. fibra dell'università), provate anche 50!
MAX_WORKERS = 20 

# Crea la cartella se non esiste
os.makedirs(OUTPUT_DIR, exist_ok=True)

# --- 2. FUNZIONE DI DOWNLOAD SINGOLO ---
def download_image(row):
    asset_id = row['product_asset_id']
    url = row['product_image_url']
    
    # Salviamo l'immagine usando il suo ID come nome (es. 12345.jpg)
    save_path = os.path.join(OUTPUT_DIR, f"{asset_id}.jpg")
    
    # TRUCCO SALVA-VITA: Se l'immagine esiste già, la salta. 
    # Così se si stacca internet e devi riavviare lo script, non ricominci da capo!
    if os.path.exists(save_path):
        return True
        
    try:
        # Timeout di 10 secondi per evitare che lo script si incanti su server lenti
        response = requests.get(url, timeout=10)
        response.raise_for_status() # Controlla che non ci siano errori 404
        
        with open(save_path, 'wb') as f:
            f.write(response.content)
        return True
    except Exception as e:
        # Se c'è un errore (es. link rotto), lo ignora silenziosamente per non fermare tutto
        return False

# --- 3. ESECUZIONE MULTITHREADING ---
def main():
    print(f"Caricamento dataset da {CSV_PATH}...")
    
    # Legge il CSV. (Aggiungi sep=';' se il file è separato da punti e virgola)
    df = pd.read_csv(CSV_PATH)
    
    # Pulizia: Rimuove eventuali righe che non hanno un URL valido
    df = df.dropna(subset=['product_image_url'])
    print(f"Trovate {len(df)} immagini da scaricare.")
    
    success_count = 0
    
    # Avvia il "motore" dei download paralleli
    with ThreadPoolExecutor(max_workers=MAX_WORKERS) as executor:
        # Assegna i compiti
        futures = {executor.submit(download_image, row): row for _, row in df.iterrows()}
        
        # tqdm genera la barra di caricamento figa a schermo
        for future in tqdm(as_completed(futures), total=len(futures), desc="Download in corso"):
            if future.result():
                success_count += 1
                
    print(f"\n🚀 Download completato! {success_count}/{len(df)} immagini salvate in: {OUTPUT_DIR}")

if __name__ == "__main__":
    main()